# **Setup Environment**

In [ ]:
# Change directory to the project root so that relative paths in .env 
# (e.g., Redis db path) resolve correctly
%cd ../

In [ ]:
import sys
sys.path.append('.')

import time
from functools import wraps
from typing import Any, Callable, Awaitable

from app.db.services import MockAsyncService

## **Initialize**

In [ ]:
mock_service = MockAsyncService()

In [ ]:
def timing_async(func: Callable[..., Awaitable[Any]]) -> Callable[..., Awaitable[Any]]:
    """
    Measure and print the execution time of an async function.

    Parameters
    ----------
    func : Callable[..., Awaitable[Any]]
        The async function to measure.

    Returns
    -------
    Callable[..., Awaitable[Any]]
        A wrapper that prints execution time before returning the result.
    """

    @wraps(func)
    async def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = await func(*args, **kwargs)
        end = time.perf_counter()
        print(f"{func.__qualname__} took {(end - start) * 1000:.2f} ms")
        return result
    return wrapper

## **Wrap instance methods with timing decorator**

In [ ]:
original_run_prediction = mock_service.run_prediction
mock_service.run_prediction = timing_async(original_run_prediction)

original_get_user = mock_service.get_user
mock_service.get_user = timing_async(original_get_user)

original_update_user = mock_service.update_user
mock_service.update_user = timing_async(original_update_user)

# **Test**

## **I. Test caching of `run_prediction()` method**

In [ ]:
await mock_service.run_prediction(model_id="id", input_data=[3,1,4,1,5])

In [ ]:
await mock_service.run_prediction(model_id="id", input_data=[3,1,4,1,5])

In [ ]:
# different inputs cause slow execution
await mock_service.run_prediction(model_id="id", input_data=[3,2,4,1,5])

## **II. Test caching of `get_user()` method**

In [ ]:
await mock_service.get_user(user_id=7)

In [ ]:
await mock_service.get_user(user_id=7)

In [ ]:
# after waiting for REDIS_TTL_FAST to expire and calling the method again with the same inputs, it executes slowly
await mock_service.get_user(user_id=7)

## **III. Test invalidating cache of `get_user()` method**

In [ ]:
await mock_service.update_user(user_id=7)

In [ ]:
# after invalidating the cache of `get_user()`, calling it again with the same inputs triggers slow execution again
await mock_service.get_user(user_id=7)